In [80]:
import os
import pandas
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib

In [64]:
test_df = pd.read_csv("../data/test.csv")
train_df = pd.read_csv("../data/train.csv")

In [65]:
train_df.info()
print("\n")
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   order_id                      45593 non-null  object 
 1    delivery_person_id           45593 non-null  object 
 2    delivery_person_age          45593 non-null  object 
 3    delivery_person_ratings      45593 non-null  object 
 4    restaurant_latitude          45593 non-null  float64
 5    restaurant_longitude         45593 non-null  float64
 6    delivery_location_latitude   45593 non-null  float64
 7    delivery_location_longitude  45593 non-null  float64
 8    order_date                   45593 non-null  object 
 9    time_ordered                 45593 non-null  object 
 10   time_order_picked            45593 non-null  object 
 11   weather_conditions           45593 non-null  object 
 12   road_traffic_density         45593 non-null  object 
 13   

In [66]:
# Too many `object` types to normalize easily, we have to figure out how to handle them

train_object_cols = train_df.select_dtypes(include=["object"])
test_object_cols = test_df.select_dtypes(include=["object"])

for col in train_object_cols:
	print(f"Train - Column: {col}, Unique Values: {train_df[col].nunique()}")
	print(train_df[col].value_counts())
	print("\n")

for col in test_object_cols:
	print(f"Test - Column: {col}, Unique Values: {test_df[col].nunique()}")
	print(test_df[col].value_counts())
	print("\n")

Train - Column: order_id, Unique Values: 45593
order_id
0x4607     1
0xb379     1
0x5d6d     1
0x7a6a     1
0x70a2     1
          ..
0x7c09     1
0xd641     1
0x4f8d     1
0x5eee     1
0x5fb2     1
Name: count, Length: 45593, dtype: int64


Train - Column:  delivery_person_id, Unique Values: 1320
 delivery_person_id
PUNERES01DEL01       67
JAPRES11DEL02        67
VADRES08DEL02        66
RANCHIRES02DEL01     66
HYDRES04DEL02        66
                     ..
BHPRES15DEL03         7
GOARES01DEL03         7
KOLRES09DEL03         6
KOCRES16DEL03         6
BHPRES010DEL03        5
Name: count, Length: 1320, dtype: int64


Train - Column:  delivery_person_age, Unique Values: 23
 delivery_person_age
35      2262
36      2260
37      2227
30      2226
38      2219
24      2210
32      2202
22      2196
29      2191
33      2187
28      2179
25      2174
34      2166
26      2159
21      2153
27      2150
39      2144
20      2136
31      2120
23      2087
NaN     1854
50        53
15        38

In [86]:
# We have 2 different datasets with one of them lacking the target column, so the logic is extracted into a function for DRY purposes
def process_data(df: pd.DataFrame) -> pd.DataFrame:
	# Copy to avoid modifying the original DataFrame
	df_processed = df.copy()

	# Strip spaces from column names
	df_processed.columns = df_processed.columns.str.strip()

	# There are some string entries that have leading/trailing spaces, we remove them. Especially important before the NaN replacement, as those do not get replaced otherwise.
	for column in df_processed.columns:
		if df_processed[column].dtype == "object":
			df_processed[column] = df_processed[column].str.strip()

	# Get rid of string nan values
	nan_strings = ["NaN", "conditions NaN"]
	df_processed.replace(nan_strings, np.nan, inplace=True)

	coord_cols = ["restaurant_latitude", "restaurant_longitude", "delivery_location_latitude", "delivery_location_longitude"]
	# Some latitude/longitude values are 0. This is likely a data quality issue as well, most likely from geocoding failures.
	# According to https://en.wikipedia.org/wiki/Null_Island it's common to use (0,0) as a placeholder for missing lat/lon values.
	# We can replace the values at and around (0,0) with NaN.
	zero_threshold = 0.3

	# These are already numeric in the dataset, but just in case we convert them to numeric and coerce errors to NaN
	# We also take absolute values to mitigate issues with negative values in lat/lon as it's unlikely that a food delivery would cross hemispheres
	for column in coord_cols:
		df_processed[column] = pd.to_numeric(df_processed[column], errors='coerce').abs()

	# We can't do this in a for loop because we need to check both latitude and longitude at the same time
	df_processed.loc[
		(df_processed["restaurant_latitude"].abs() < zero_threshold) & (df_processed["restaurant_longitude"].abs() < zero_threshold),
		coord_cols
	] = np.nan
	df_processed.loc[
		(df_processed["delivery_location_latitude"].abs() < zero_threshold) & (df_processed["delivery_location_longitude"].abs() < zero_threshold),
		coord_cols
	] = np.nan

	# `conditions ` suffix is redundant in weather_conditions, as it's implied by the column name.
	# We remove it so the column name after one-hot encoding is cleaner.
	df_processed["weather_conditions"] = df_processed["weather_conditions"].str.replace("conditions ", "", regex=False)

	# Extract numeric part from time_taken_minutes and convert to float for the training df
	if "time_taken_minutes" in df_processed.columns:
		df_processed["time_taken_minutes"] = df_processed["time_taken_minutes"].str.extract(r"(\d+)").astype(float)

	# Some numeric columns are stored as strings, we convert them to numeric types
	numeric_cols = ["delivery_person_age", "delivery_person_ratings", "multiple_deliveries"]
	for column in numeric_cols:
		if column in df_processed.columns:
			df_processed[column] = pd.to_numeric(df_processed[column], errors="coerce")

	df_processed["order_date"] = pd.to_datetime(df_processed["order_date"], format="%d-%m-%Y", errors="coerce")

	# Impute missing values that we use for feature engineering. The rest will be handled in the pipeline later.
	df_processed.fillna({"order_date": df_processed["order_date"].mode()[0]}, inplace=True)
	df_processed.fillna({"time_ordered": df_processed["time_ordered"].mode()[0]}, inplace=True)
	df_processed.fillna({"time_order_picked": df_processed["time_order_picked"].mode()[0]}, inplace=True)

	# Convert time columns to datetime
	df_processed["order_datetime"] = pd.to_datetime(df_processed["order_date"].astype(str) + " " + df_processed["time_ordered"])
	df_processed["picked_datetime"] = pd.to_datetime(df_processed["order_date"].astype(str) + " " + df_processed["time_order_picked"])

	# --- Feature engineering ---

	# Distance-based features

	# There's a small problem with the latitude/longitude columns. We can't really use them as-is, because the data is localized to a specific city/region.
	# The model trained from them wouldn't generalize well to other locations.
	# To solve this, we can calculate the distance between the restaurant and the delivery.
	# Source of the formula: https://stackoverflow.com/questions/27928/calculate-distance-between-two-latitude-longitude-points-haversine-formula

	def haversine_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
		# Our input is in degrees, we need to convert it to radians
		lat1_rad, lon1_rad = np.radians(lat1), np.radians(lon1)
		lat2_rad, lon2_rad = np.radians(lat2), np.radians(lon2)
		dlat = lat2_rad - lat1_rad
		dlon = lon2_rad - lon1_rad
		a = (np.sin(dlat / 2) ** 2) + (np.cos(lat1_rad) * np.cos(lat2_rad) * (np.sin(dlon / 2) ** 2))
		r = 6371  # Earth's radius in kilometers
		c = 2 * np.atan2(np.sqrt(a), np.sqrt(1 - a))
		return r * c

	df_processed["distance_km"] = haversine_distance(
		df_processed["restaurant_latitude"], df_processed["restaurant_longitude"],
		df_processed["delivery_location_latitude"], df_processed["delivery_location_longitude"]
	)

	# We have some NaN values we filled in earlier, so we handle them here by imputing the median distance
	distance_median = df_processed["distance_km"].median()
	df_processed.fillna({"distance_km": distance_median}, inplace=True)

	# Time-based features
	df_processed["time_to_pickup_minutes"] = (df_processed["picked_datetime"] - df_processed["order_datetime"]).dt.total_seconds() / 60.0
	df_processed["order_hour"] = df_processed["order_datetime"].dt.hour
	df_processed["order_day_of_week"] = df_processed["order_datetime"].dt.dayofweek  # Indices based on monday=0, sunday=6
	# We can't really use day of month or year as features because our data is from 2-3 months in 2022 only, so it wouldn't generalize well

	# Cyclical features for hour and day of week
	df_processed["order_hour_sin"] = np.sin(2 * np.pi * df_processed["order_hour"] / 24)
	df_processed["order_hour_cos"] = np.cos(2 * np.pi * df_processed["order_hour"] / 24)
	df_processed["order_day_of_week_sin"] = np.sin(2 * np.pi * df_processed["order_day_of_week"] / 7)
	df_processed["order_day_of_week_cos"] = np.cos(2 * np.pi * df_processed["order_day_of_week"] / 7)

	# Since traffic is ordinal, we can map it to integers
	traffic_mapping = {
		"Low": 1,
		"Medium": 2,
		"High": 3,
		"Jam": 4
	}
	df_processed["road_traffic_density"] = df_processed["road_traffic_density"].map(traffic_mapping)

	# We can drop the columns we used to create the features
	cols_to_drop = [
		"order_id", "delivery_person_id",  # IDs
		"restaurant_latitude", "restaurant_longitude", "delivery_location_latitude", "delivery_location_longitude",  # Raw lat/lon values
		"time_ordered", "time_order_picked", "order_date", "order_datetime", "picked_datetime",  # Raw time values
		"order_hour", "order_day_of_week"  # We have cyclical features for these now
	]
	df_processed = df_processed.drop(columns=cols_to_drop, errors="ignore")

	return df_processed

In [88]:
test_df_processed = process_data(test_df)
train_df_processed = process_data(train_df)

# Split features and target
y_train = train_df_processed["time_taken_minutes"]
X_train = train_df_processed.drop(columns=["time_taken_minutes"])
X_test = test_df_processed

# Align columns in test set to match train set
train_cols = X_train.columns
test_cols = X_test.columns
X_train, X_test = X_train.align(X_test, join="inner", axis=1)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (45593, 16), y_train shape: (45593,)
X_test shape: (11399, 16)


In [89]:
numerical_cols = X_train.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X_train.select_dtypes(include=["object"]).columns.tolist()

In [90]:
numerical_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="median")),
	("scaler", StandardScaler())
], memory=None)

categorical_transformer = Pipeline(steps=[
	("imputer", SimpleImputer(strategy="most_frequent")),
	("onehot", OneHotEncoder(handle_unknown="ignore", drop="first"))
], memory=None)

preprocessor = ColumnTransformer(
	transformers=[
		("num", numerical_transformer, numerical_cols),
		("cat", categorical_transformer, categorical_cols)
	],
	remainder="passthrough",
)

print("Fitting the preprocessor...")
preprocessor.fit(X_train)
print("Preprocessor fitted.")

Fitting the preprocessor...
Preprocessor fitted.


In [91]:
X_train_final = preprocessor.transform(X_train)
X_test_final = preprocessor.transform(X_test)
print("Shape of final training features:", X_train_final.shape)
print("Shape of final testing features:", X_test_final.shape)
print("Shape of final training target:", y_train.shape)

Shape of final training features: (45593, 25)
Shape of final testing features: (11399, 25)
Shape of final training target: (45593,)


In [92]:
os.makedirs("../processed", exist_ok=True)
joblib.dump(preprocessor, "../processed/preprocessor.joblib")
np.save("../processed/X_train_final.npy", X_train_final)
np.save("../processed/X_test_final.npy", X_test_final)
y_train.to_csv("../processed/y_train.csv", index=False, header=True)  # This doesn't have to be a numpy array